In [1]:
import json
from difflib import get_close_matches
import math
import sys
import matplotlib.pyplot as plt
import datetime as dt

In [3]:
# This libaries is for carrying out Chat Gpt know

import openai as gpt
import gradio as gr

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ANSI codes for colors
class Color:
    RESET = "\033[0m"
    RED = "\033[91m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    BLUE = "\033[94m"
    PURPLE = "\033[95m"
    CYAN = "\033[96m"
    WHITE = "\033[97m"

In [14]:
def load_oil_well_db (file_path: str):
    with open (file_path, "r") as file:
        data: dict = json.load(file)
    return data

def save_oil_well_db (file_path: str, data: dict):
    with open(file_path, "w") as file:
        json.dump(data, file, indent = 2)
        
def find_best_match(user_question: str, questions: list[str]):
    matches: list = get_close_matches(user_question, questions, n=1, cutoff=0.6)
    return matches[0] if matches else None

def get_answer_question(question:str, oil_well_chatbot_db: dict):
    for q in oil_well_chatbot_db["questions"]:
        if q["question"] == question:
            return q["answer"]
        
oil_well_chatbot_db = load_oil_well_db("oil_well_chatbot_db.json")

# Main Menu Function
def main_menu (oil_well_chatbot_db: dict):
    print (Color.CYAN + '''\n Welcome to Blo Simulator and Interaction Bot.
           
    Choose from the Menu Below: ''' + Color.RESET)
    
    menu = oil_well_chatbot_db["menu"]
    [print(f"{n})\t{m.title()}") for n,m in menu.items()]
    print("\n\n")
    choices = input()
    return choices

# Input Parameters Function
def input_parameters():
    print(Color.YELLOW + "Enter the following parameters for the well\n" + Color.RESET)
    try:
        ipr = float(input("Initial Production Rate"))
        decline_rate = float(input("Decline Rate"))
        time = int(input("Time (in years)"))
        oiip = float (input("Oil Originally in Place"))
    except:
        print(Color.RED + "Enter a valid number and not a character or alphabet" + Color.RESET)
        sys.exit()

    return ipr, decline_rate, time, oiip

# the arps model and execution functions
# ipr - initial production rate, cpr - current production rate
def arps_model(ipr, decline_rate, time):
    cpr = ipr * math.exp (-decline_rate * time )
    return cpr
def simulation (ipr, decline_rate, simulation_time):
    production_rate = []
    time_points = []
    for i in range (1, simulation_time + 1):
        cpr = arps_model(ipr, decline_rate, i)
        production_rate.append(cpr)
        time_points.append(i)

    return production_rate, time_points

def analysis(ipr, decline_rate, oiip, simulation_time):
    print (Color.YELLOW + f"Below are the Input Parameters of the well \n\n" +
           f"Initial Production Rate = {ipr}\n" +
           f"Decline Production Rate = {decline_rate} \nTime (in years) = {simulation_time}" + Color.RESET)

    prod_rate, time_points = simulation(ipr, decline_rate, simulation_time)
    prod_sum = sum(prod_rate)
    ave_prod = prod_sum / len(prod_rate)
    rec_factor = (ave_prod / oiip) * 100
    print(Color.GREEN + f"\nCummulative Production of the Well = {prod_sum}\n" +
          f"Average Production of the well = {ave_prod}\n" +
          f"Recovery Factor of the well = {rec_factor}\n" + Color.RESET)
def visualise(time_points, production_data):
    plt.plot(production_data, time_points)
    plt.xlabel("Simulation Time")
    plt.ylabel("Production rate")
    plt.title("Plot Showing the Decline of Production Rate of Well")
    plt.show()
def exit_logic():
    while True:
        ans = input(Color.RED + "Do you want to perform any other operations? (y/n)" + Color.RESET)
        if ans.lower() == "y" or ans.lower() == "n":
            break
        else:
            print("Enter valid response")
    return ans

# Chat Buddy Function capable of learning and responding
# It generates answers based on what people have asked in the past.
# Learns when it cannot find answers to questions
def chat_bot():
    oil_well_chatbot_db: dict = load_oil_well_db("oil_well_chatbot_db.json")
    
    print(Color.YELLOW + ''' Hello I am Blo. 
    your chat buddy and for knowledge sharig. 
    you can ask questions and teach me if I don't give you required answers.
          ''' + Color.RESET)
    
    while True:
        user_input: str = input("You ('quit' to exit): ")
        
        if user_input.lower() == "quit":
            print("\nThanks for the chat. See you soon")
            break
        
        best_match: str | None = find_best_match(user_input, [q["question"] for q in oil_well_chatbot_db["questions"]])
        
        if best_match:
            answer: str = get_answer_question(best_match, oil_well_chatbot_db)
            print(Color.GREEN + f"Bot: {answer}" + Color.RESET)
        else:
            print(Color.GREEN + "Bot: I don't know the answer. Can you teach me?" + Color.RESET)
            new_answer: str = input(Color.YELLOW + "Type the answer or 'skip' to skip: " + Color.RESET)
            
            if new_answer.lower() != "skip":
                oil_well_chatbot_db["questions"].append({"question": user_input, "answer": new_answer})
                save_oil_well_db("oil_well_chatbot_db.json", oil_well_chatbot_db)
                print(Color.GREEN + "Bot: Thank You! I have learnt a new response." + Color.RESET)

# function to send prompt and receive response from GPT or any other LLM
def gpt_room(user_input, key):
    gpt.api_key = key
    messages = [{"role": "system", "content":"Virtual Assistant"}]
    messages.append({"role":"system", "content":user_input})
    response = gpt.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = messages
    )
    chatgpt_reply = response["choices"][0]["message"]["content"]
    oil_well_chatbot_db["gpt_questions"].append({"prompt":messages[-1]["content"], "response":chatgpt_reply})
    print("\n" + chatgpt_reply + "\n")
    return chatgpt_reply

# function to execute the gpt_room and print response
def chat_gpt_room():
    print("Welcome to the Oil Simulator Ai Chat Room Powered by Chat GPT \n")
    key = None
    while True:
        if key == None:
            print("There is no API key available. So function is currently unavailable.\n Do you have your own API Key to test? ")
            ans = input("Enter y/n")
            
            if ans.lower() == "y":
                key = input("Enter key ")
        
            else:
                break
    
        user_input = None
        print("Enter your prompt below\n")
        user_input = input("Enter Prompt ")
        
        if user_input == "quit":
            break   
        try:
            gpt_room(user_input)
        except:
            print("Issue getting response. Check Connection or API key")
        #demo = gr.Interface(fn=gpt_room, inputs = "text", outputs = "text", title = "Real Estate Pro")
        #demo.launch()
 
# teach the program new mathematical functions to perform on data   
def teach_math_operation():
    print("Sorry Function Not Available at the moment\n Work in Progress")
    #formula = input("Enter formula using lambda")
    #values = input("Input a list of values")
    #soln = formula (*values)
    
    #return soln
  
# store a report of a Well in the database  
def well_write_report(oil_well_chatbot_db):

    while True:
        print(Color.CYAN + '''  
    Do you want to enter the data of a new well or an existing one?
    (new/existing)''' + Color.CYAN)
    
        status = input(Color.YELLOW + "Enter answer (or 'quit' to return to menu)" + Color.RESET)
        
        if status.lower() == "quit":
            break
    
        elif status.lower() == "new":
            well_name = input("Enter the name of the well")
            for k in oil_well_chatbot_db["oil_well_report"]:
                if k[0] == well_name:
                    print(Color.BLUE + "Well already exists.\n You can modify the existing data" + Color.RESET)
                    break
            else:
                i_report = input("Enter the report of the well")
                report_date = str(dt.date.today())
                report = [report_date, i_report]
                oil_well_chatbot_db["oil_well_report"].append([well_name, report])
                save_oil_well_db("oil_well_chatbot_db.json", oil_well_chatbot_db)
                
                print(Color.GREEN + ''' Report saved!
                Do you want to save another? (y/n)\n''' + Color.RESET)
                ans = input(Color.YELLOW + "Enter answer" + Color.RESET)
                if ans.lower() == "y":
                    continue
                elif ans.lower() == "n":
                    break
                else:
                    print(Color.RED + "Invalid Answer. Quitting Interface" + Color.RED)
                    break
    
        elif status.lower() == "existing":
            print(Color.YELLOW + "Enter the name of the well" + Color.RESET)
            existing_well = input()
            well_status = 0
            
            for k in oil_well_chatbot_db["oil_well_report"]:
                if k[0] == existing_well:
                    well_status += 1
                    print("Enter a new report to add to the previous one")
                    new_report = input()
                    report_date = str(dt.date.today())
                    dated_report = [report_date, new_report]
                    k.append(dated_report)
                    save_oil_well_db("oil_well_chatbot_db.json", oil_well_chatbot_db)
                    print(f"New Report saved successfully for {existing_well}")
                    break
                    
                elif well_status == 0:
                    print(Color.YELLOW + "Well is non-existent.\n consider entering new report" + Color.RESET)
    
        else:
            print(Color.RED + "Enter a valid response or 'quit'" + Color.RESET)
 
# read a stored report in the database           
def well_read_report(oil_well_chatbot_db):
    
    print('''Welcome! All Reports are duely saved.
    Note that they were saved with their dates shown as the fist element''')
    
    while True:
        print("Enter the name of the well you wish to Read")
    
        status = input("Enter answer (or 'quit' to return to menu)")
        
        if status.lower() == "quit":
            break
        
        well_status = 0
        for k in oil_well_chatbot_db["oil_well_report"]:
            if k[0] == status.lower():
                well_status = 1
                print(f"{k[0]} Report")
                for i in range(1, len(k)):
                    print(f"Date:\t{k[i][0]} \n \n")
                    print(f"Report: \n{k[i][1]} \n\n")
                break
            
            elif well_status == 0:
                print(f"No Report found for {status} well")
                
        print("Do you want to read more report?\n (y/n)")
        ans = input()
        if ans == "y":
            continue
        else:
            break

# executes the read and write report functions
def well_report(oil_well_chatbot_db):
    print(Color.CYAN + " ****** Welcome to the Report Database *****" + Color.RESET)
    print(''' Do you want to?
          1) Enter a New Report (write)
          2) Read an Existing Report''')
    print(Color.YELLOW + "\nEnter 'Write' or 'Read' " + Color.RESET)
    
    ans = input()
    if ans.lower() == "write" or ans == "1":
        well_write_report(oil_well_chatbot_db)
        
    elif ans.lower() == "read" or ans == "2":
        well_read_report(oil_well_chatbot_db)

# Executes the whole functions into one program
def well_manager(oil_well_chatbot_db:dict):
    while True:
        choice = main_menu(oil_well_chatbot_db)
        if choice == "1" or choice.lower() == "input parameters":
            ipr, decline_rate, time, oiip = input_parameters()
            print(Color.BLUE + "Do you want to save well name and details to database? (y/n)" + Color.BLUE)
            choice_response = input()
            if choice_response == "y":
                well_name = input("Enter the name of well: ")
                oil_well_chatbot_db['wells'].append({"well_name": well_name, "ipr": ipr, "decline_rate":decline_rate, "time":time, "oiip":oiip})
                save_oil_well_db("oil_well_chatbot_db.json", oil_well_chatbot_db)
                print("Well Details saved successfully")
                
        elif choice == "2" or choice.lower() == "simulate wells":
            try:
                prod_data, time_points = simulation (ipr, decline_rate, time)
                print(prod_data)
            except:
                print("Enter Input Parameters from Menue First")
                
        elif choice == "3" or choice.lower() == "analyse production":
            try:
                analysis(ipr, decline_rate, oiip, time)
            except:
                print("Enter Input Parameters from Menue First")

        elif choice == "4" or choice.lower() == "visualise result":
            try:
                visualise (prod_data, time_points)
            except:
                print("Issues with data. Run a simulation first")
                
        elif choice == "5" or choice.lower() == "teach and interact":
            if __name__ == '__main__':
                chat_bot()
        
        elif choice == "6" or choice.lower() == "teach math operation":
            try:
                teach_math_operation()
            except:
                print("Enter Input Parameters from Menue First")
        
        elif choice == "7" or choice.lower() == "chat gpt room":
            try:
                chat_gpt_room()
            except:
                print("Enter Input Parameters from Menue First")
        
        elif choice == "8" or choice.lower() == "well report":
            try:
                well_report(oil_well_chatbot_db)
            except:
                print("Enter Input Parameters from Menue First")
        
        elif choice == "9" or choice.lower() == "exit":
            ans = exit_logic ()
            if ans == "y":
                continue
            else:
                break


                          

In [15]:
well_manager(oil_well_chatbot_db)


 Welcome to Blo Simulator and Interaction Bot.
           
    Choose from the Menu Below: 
1)	Input Parameters
2)	Simulate Wells
3)	Analyse Production
4)	Visualise Result
5)	Teach And Interact
6)	Teach Math Operation
7)	Chat Gpt Room
8)	Well Report
9)	Exit



Welcome to the Oil Simulator Ai Chat Room Powered by Chat GPT 

There is no API key available. So function is currently unavailable.
 Do you have your own API Key to test? 

 Welcome to Blo Simulator and Interaction Bot.
           
    Choose from the Menu Below: 
1)	Input Parameters
2)	Simulate Wells
3)	Analyse Production
4)	Visualise Result
5)	Teach And Interact
6)	Teach Math Operation
7)	Chat Gpt Room
8)	Well Report
9)	Exit





In [82]:
def teach_math_operation():
    print("Sorry Function Not Available at the moment\n Work in Progress")
    #formula = input("Enter formula using lambda")
    #values = input("Input a list of values")
    #soln = formula (*values)
    
    #return soln

In [83]:
teach_math_operation()

TypeError: 'str' object is not callable